<a href="https://colab.research.google.com/github/kidrono/Kaggle-competitions/blob/main/kaggle_disaster_tweets.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Kaggle competition: Natural Language Processing with Disaster Tweets
The competition information could be found [here](https://www.kaggle.com/competitions/nlp-getting-started/overview)

In [31]:
! pip install kaggle

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [32]:
! mkdir ~/.kaggle

mkdir: cannot create directory ‘/root/.kaggle’: File exists


In [33]:
! cp kaggle.json ~/.kaggle/

In [34]:
! chmod 600 ~/.kaggle/kaggle.json

In [35]:
! kaggle competitions download nlp-getting-started

nlp-getting-started.zip: Skipping, found more recently modified local copy (use --force to force download)


In [36]:
import zipfile
import os

if os.path.isfile("/data/train.csv"):
  print("allready exists")
else:
  zip_ref = zipfile.ZipFile("nlp-getting-started.zip", 'r')
  zip_ref.extractall("/data")
  zip_ref.close()

allready exists


In [37]:
import pandas as pd
import numpy as np
import re
import torch

In [38]:
train_df = pd.read_csv("/data/train.csv")
train_labels = train_df['target']
test_df = pd.read_csv("/data/test.csv")

In [39]:
# one_hot_dict = dict()
# i = 0
# for text in train_df['text']:
#   clean_tweet = re.sub(r'[^a-zA-Z ]+', '', text)
#   tweet = clean_tweet.split(' ')
#   for word in tweet:
#     word = word.lower()
#     if word not in one_hot_dict.keys():
#       one_hot_dict[word] = i
#       i += 1
# vec_dim = len(one_hot_dict) + 1
# vec_dim

In [40]:
# def sentence_vector(text):
#   vec = np.zeros(vec_dim)
#   clean_tweet = re.sub(r'[^a-zA-Z ]+', '', text)
#   tweet = clean_tweet.split(' ')
#   for word in tweet:
#     word = word.lower()
#     if word in one_hot_dict.keys():
#       vec[one_hot_dict[word]] += 1
#     else:
#       vec[-1] += 1
#   return vec

In [41]:
# X = []
# for text in train_df['text']:
#   X.append(sentence_vector(text))
# X = np.array(X)
# X.shape

In [42]:
class LogisticRegression(torch.nn.Module):

  def __init__(self, input_dim, classes) -> None:
    super().__init__()
    self.linear = torch.nn.Linear(input_dim, classes)
    self.sigmoid = torch.nn.Sigmoid()
  
  def forward(self, x):
    x = self.linear(x)
    x = self.sigmoid(x)
    return x

In [43]:
def train(x, y, model, loss, optimizer, ephocs):
  for epoch in epochs:
    y_pred = model(x)
    pred_loss = loss(y, y_pred)
    optimizer.zero_grad()
    loss.backward()
    optimizer.step()
    


In [44]:
from gensim.models import Word2Vec
import gensim

In [45]:
# cleaned_list = []
# i = 0
# for text in train_df['text']:
#   clean_tweet = re.sub(r'[^a-zA-Z ]+', '', text)
#   tweet = clean_tweet.lower()
#   final = tweet.encode('utf-8').split()
#   cleaned_list.append(final)
# train_df['clean_tweet'] = cleaned_list
# train_df


In [46]:
train_df['text_clean'] = train_df['text'].apply(lambda x: gensim.utils.simple_preprocess(x))
test_df['text_clean'] = test_df['text'].apply(lambda x : gensim.utils.simple_preprocess(x))

In [47]:
train_embedding_w2v = Word2Vec(train_df['text_clean'], min_count=1,
                              vector_size=300, window=5)

In [48]:
train_embedding_w2v.wv.most_similar('fire')

[('new', 0.9999236464500427),
 ('for', 0.9999111890792847),
 ('today', 0.9999054074287415),
 ('storm', 0.9999005198478699),
 ('police', 0.9998998045921326),
 ('near', 0.9998946785926819),
 ('at', 0.9998900890350342),
 ('from', 0.9998883008956909),
 ('of', 0.9998878836631775),
 ('accident', 0.9998804330825806)]

In [49]:
X_train = train_df['text_clean']
words = set(train_embedding_w2v.wv.index_to_key )
X_train_vect = np.array([np.array([train_embedding_w2v.wv[i] for i in ls if i in words])
                         for ls in X_train])
test_vect = np.array([np.array([train_embedding_w2v.wv[i] for i in ls if i in words])
                         for ls in test_df['text_clean']])

<ipython-input-49-973befb3b90e>:3: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.
  X_train_vect = np.array([np.array([train_embedding_w2v.wv[i] for i in ls if i in words])
<ipython-input-49-973befb3b90e>:5: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.
  test_vect = np.array([np.array([train_embedding_w2v.wv[i] for i in ls if i in words])


In [70]:
X_train_vect_avg = []
for v in X_train_vect:
    if v.size:
        X_train_vect_avg.append(v.mean(axis=0))
    else:
        X_train_vect_avg.append(np.zeros(100, dtype=float))
X_train_vect_avg[0].shape 

(300,)

In [93]:
test_vect_avg = []
for v in test_vect:
    if v.size:
        test_vect_avg.append(v.mean(axis=0))
    else:
        test_vect_avg.append(np.zeros(100, dtype=float))
test_vect_avg[0].shape

(300,)

In [88]:
test_df['text_clean'][3015]

['ashniggas', 'tink']

In [97]:
i = 0
zeros = []
for avg_vec in test_vect_avg:
  if avg_vec.size != 300:
    zeros.append(i)
    print(i, avg_vec.size)
    print(test_vect_avg[i])
  i += 1

for i in zeros:
  test_vect_avg[i] = np.zeros(300)

In [54]:
from sklearn.linear_model import LogisticRegression

model = LogisticRegression(penalty='none').fit(X_train_vect_avg, train_labels)
y_prob= model.predict_proba(X_train_vect_avg)[:, 1]
y_pred = np.round(y_prob)
acc = np.sum(y_pred == train_labels) / len(train_labels)
acc

/usr/local/lib/python3.10/dist-packages/sklearn/linear_model/_logistic.py:1173: FutureWarning: `penalty='none'`has been deprecated in 1.2 and will be removed in 1.4. To keep the past behaviour, set `penalty=None`.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/sklearn/linear_model/_logistic.py:458: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


0.7265204255878103

In [115]:
y_test_prob = model.predict_proba(test_vect_avg)[:, 1]
y_pred_test = np.round(y_test_prob).astype(int)


In [116]:
type(y_pred_test[0])

numpy.int64

In [117]:
test_df['target'] = y_pred_test
test_df.head()

,id,keyword,location,text,text_clean,target
0,0,NaN,NaN,Just happened a terrible car crash,"[just, happened, terrible, car, crash]",1
1,2,NaN,NaN,"Heard about #earthquake is different cities, s...","[heard, about, earthquake, is, different, citi...",0
2,3,NaN,NaN,"there is a forest fire at spot pond, geese are...","[there, is, forest, fire, at, spot, pond, gees...",1
3,9,NaN,NaN,Apocalypse lighting. #Spokane #wildfires,"[apocalypse, lighting, spokane, wildfires]",0
4,11,NaN,NaN,Typhoon Soudelor kills 28 in China and Taiwan,"[typhoon, soudelor, kills, in, china, and, tai...",1


In [118]:
submission_df = test_df[['id', 'target']]

submission_df.to_csv("submission1.csv", index=False)
